# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [ ]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([

    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))  # Normalize to [-1, 1]
])  # BUG normalize

loader=DataLoader(torchvision.datasets.MNIST('./data',True,download=True,transform=transform),batch_size=256,num_workers=2,shuffle=True)
z_dim=100
g_lr=2e-2
d_lr=2e-5


In [ ]:

class Reshape(nn.Module):
  """A custom reshape layer."""
  def __init__(self, shape):
    super(Reshape, self).__init__()
    self.shape = shape

  def forward(self, x):
    return x.view(*self.shape)

In [ ]:
class D(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(32,64,4,2,1),
                               nn.BatchNorm2d(64),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(64,1,7,1,0))

    def forward(self,x):
      return self.net(x).view(x.size(0),-1)

class G(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        self.z_dim=z_dim
        self.net=nn.Sequential(
            nn.Linear(z_dim, 64*4*4),
            nn.BatchNorm1d(64*4*4),
            nn.ReLU(True),

            Reshape((-1, 64, 4, 4)),

            nn.ConvTranspose2d(64,128,4,1,0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,1,4,2,1),
            nn.Tanh())  # BUG missing Tanh
    def forward(self,z):

      z=z.view(z.size(0),-1)

      return self.net(z)  # BUG reshape

In [ ]:

Dnet=D().to(device)
Gnet=G(z_dim).to(device)
crit=nn.BCEWithLogitsLoss()
opt_d=torch.optim.Adam(Dnet.parameters(),lr=d_lr,betas=(0.9,0.999))
opt_g=torch.optim.Adam(Gnet.parameters(),lr=g_lr,betas=(0.9,0.999))

In [ ]:
from tqdm import tqdm
for real,_ in tqdm(loader):

    real=real.to(device)
    b=real.size(0)
    z=torch.randn(b,z_dim,device=device)
    fake=Gnet(z.view(b,z_dim,1,1))
    loss_d=crit(Dnet(real),torch.ones(b,1,device=device))+crit(Dnet(fake),torch.zeros(b,1,device=device))  # BUG labels

    loss_d.backward();
    opt_d.step()  # BUG 3
    z= torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))

    loss_g= crit(Dnet(fake),torch.zeros(b,1,device=device))  # BUG non-sat
    loss_g.backward()  # BUG missing step
    opt_g.step()

    opt_g.zero_grad()
    opt_d.zero_grad()
print('Now fix all the issues.')

100%|██████████| 235/235 [00:11<00:00, 19.94it/s]

Now fix all the issues.
